In [59]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [50]:
# Define symbolic variables for angles and dimensions
theta_1, theta_2, theta_3, theta_4, theta_5 = sp.symbols("theta_1 theta_2 theta_3 theta_4 theta_5")
d_1, d_5 = sp.symbols("d_1 d_5")  # d_5 for the last joint
a_2, a_3 = sp.symbols("a_2 a_3")  # a_2 and a_3 for the lengths of the second and third links

In [51]:
# Alpha values in degrees, with an updated value for alpha_4
alpha = [90, 0, 0, 90, 0]


In [52]:
# Helper function to create a transformation matrix from DH parameters
def dh_matrix(theta, d, a, alpha):
    alpha_rad = sp.rad(alpha)  # Convert alpha from degrees to radians
    return sp.Matrix([
        [sp.cos(theta), -sp.sin(theta) * sp.cos(alpha_rad), sp.sin(theta) * sp.sin(alpha_rad), a * sp.cos(theta)],
        [sp.sin(theta), sp.cos(theta) * sp.cos(alpha_rad), -sp.cos(theta) * sp.sin(alpha_rad), a * sp.sin(theta)],
        [0, sp.sin(alpha_rad), sp.cos(alpha_rad), d],
        [0, 0, 0, 1]
    ])

In [53]:
# Create transformation matrices for each joint using the updated parameters
A1 = dh_matrix(theta_1, d_1, 0, alpha[0])
A2 = dh_matrix(theta_2, 0, a_2, alpha[1])
A3 = dh_matrix(theta_3, 0, a_3, alpha[2])
A4 = dh_matrix(theta_4, 0, 0, alpha[3])  # a_4 is zero
A5 = dh_matrix(theta_5, d_5, 0, alpha[4])  # a_5 is zero, added d_5

In [54]:
# Compute the overall transformation matrix by multiplying individual matrices
T = A1 * A2 * A3 * A4 * A5

# Initialize pretty printing for better output readability
sp.init_printing(use_unicode=True)

# Extract the position vector from the transformation matrix
position = T[:3, 3]

In [55]:

# Compute the Jacobian for linear velocity
Jv = sp.Matrix.hstack(
    position.jacobian([theta_1, theta_2, theta_3, theta_4, theta_5])
)

# Extract the orientation part (rotation matrix) from the transformation matrix
R = T[:3, :3]

# Initialize the Jacobian for angular velocity
Jw = sp.Matrix.zeros(3, 5)

# The angular part of the Jacobian is given by the z-axis of the previous frames
z = sp.Matrix([[0], [0], [1]])  # z0 axis (base frame)

# Compute z-axes of each frame
z1 = A1[:3, :3] * z
z2 = (A1 * A2)[:3, :3] * z
z3 = (A1 * A2 * A3)[:3, :3] * z
z4 = (A1 * A2 * A3 * A4)[:3, :3] * z

# Populate the Jacobian for angular velocity
Jw[:, 0] = z[:, 0]
Jw[:, 1] = z1[:, 0]
Jw[:, 2] = z2[:, 0]
Jw[:, 3] = z3[:, 0]
Jw[:, 4] = z4[:, 0]

# Print the Jacobians
print("Jacobian for linear velocity (Jv):")
sp.pprint(Jv)

Jacobian for linear velocity (Jv):
⎡-a₂⋅sin(θ₁)⋅cos(θ₂) + a₃⋅sin(θ₁)⋅sin(θ₂)⋅sin(θ₃) - a₃⋅sin(θ₁)⋅cos(θ₂)⋅cos(θ₃)
⎢                                                                             
⎢a₂⋅cos(θ₁)⋅cos(θ₂) - a₃⋅sin(θ₂)⋅sin(θ₃)⋅cos(θ₁) + a₃⋅cos(θ₁)⋅cos(θ₂)⋅cos(θ₃) 
⎢                                                                             
⎣                                                                             

 + d₅⋅((sin(θ₁)⋅sin(θ₂)⋅sin(θ₃) - sin(θ₁)⋅cos(θ₂)⋅cos(θ₃))⋅sin(θ₄) - (sin(θ₁)⋅
                                                                              
+ d₅⋅((-sin(θ₂)⋅sin(θ₃)⋅cos(θ₁) + cos(θ₁)⋅cos(θ₂)⋅cos(θ₃))⋅sin(θ₄) - (-sin(θ₂)
                                                                              
                          0                                                   

sin(θ₂)⋅cos(θ₃) + sin(θ₁)⋅sin(θ₃)⋅cos(θ₂))⋅cos(θ₄))   -a₂⋅sin(θ₂)⋅cos(θ₁) - a₃
                                                                              
⋅cos(θ₁)⋅cos(θ₃

In [56]:
print("\nJacobian for angular velocity (Jw):")
sp.pprint(Jw)


Jacobian for angular velocity (Jw):
⎡0  sin(θ₁)   sin(θ₁)   sin(θ₁)   (-sin(θ₂)⋅sin(θ₃)⋅cos(θ₁) + cos(θ₁)⋅cos(θ₂)⋅
⎢                                                                             
⎢0  -cos(θ₁)  -cos(θ₁)  -cos(θ₁)  (-sin(θ₁)⋅sin(θ₂)⋅sin(θ₃) + sin(θ₁)⋅cos(θ₂)⋅
⎢                                                                             
⎣1     0         0         0                      -(-sin(θ₂)⋅sin(θ₃) + cos(θ₂)

cos(θ₃))⋅sin(θ₄) - (-sin(θ₂)⋅cos(θ₁)⋅cos(θ₃) - sin(θ₃)⋅cos(θ₁)⋅cos(θ₂))⋅cos(θ₄
                                                                              
cos(θ₃))⋅sin(θ₄) - (-sin(θ₁)⋅sin(θ₂)⋅cos(θ₃) - sin(θ₁)⋅sin(θ₃)⋅cos(θ₂))⋅cos(θ₄
                                                                              
⋅cos(θ₃))⋅cos(θ₄) + (sin(θ₂)⋅cos(θ₃) + sin(θ₃)⋅cos(θ₂))⋅sin(θ₄)               

)⎤
 ⎥
)⎥
 ⎥
 ⎦


In [57]:
# Check the rank of the Jacobian matrices
rank_Jv = Jv.rank()
rank_Jw = Jw.rank()

print("\nRank of the Jacobian for linear velocity (Jv):")
sp.pprint(rank_Jv)

print("\nRank of the Jacobian for angular velocity (Jw):")
sp.pprint(rank_Jw)


Rank of the Jacobian for linear velocity (Jv):
3

Rank of the Jacobian for angular velocity (Jw):
3


In [58]:
theta_values = {
    theta_1: 10,
    theta_2: 0,
    theta_3: 0,
    theta_4: 0,
    theta_5: 0,
}
Jv_sub = Jv.subs(theta_values)
Jw_sub = Jw.subs(theta_values)
rank_Jv_sub = Jv_sub.rank()
rank_Jw_sub = Jw_sub.rank()
print("\nJacobian for linear velocity (Jv) at specific configuration:")
sp.pprint(Jv_sub)
print("\nRank of the Jacobian for linear velocity (Jv) at specific configuration:")
sp.pprint(rank_Jv_sub)
print("\nJacobian for angular velocity (Jw) at specific configuration:")
sp.pprint(Jw_sub)
print("\nRank of the Jacobian for angular velocity (Jw) at specific configuration:")
sp.pprint(rank_Jw_sub)


# Check if the configuration is singular
is_singular = rank_Jv_sub+ rank_Jw_sub < 5
print(f"\nIs the configuration singular? {'Yes' if is_singular else 'No'}")



Jacobian for linear velocity (Jv) at specific configuration:
⎡-a₂⋅sin(10) - a₃⋅sin(10)  d₅⋅cos(10)  d₅⋅cos(10)  d₅⋅cos(10)  0⎤
⎢                                                               ⎥
⎢a₂⋅cos(10) + a₃⋅cos(10)   d₅⋅sin(10)  d₅⋅sin(10)  d₅⋅sin(10)  0⎥
⎢                                                               ⎥
⎣           0               a₂ + a₃        a₃          0       0⎦

Rank of the Jacobian for linear velocity (Jv) at specific configuration:
3

Jacobian for angular velocity (Jw) at specific configuration:
⎡0  sin(10)   sin(10)   sin(10)   0 ⎤
⎢                                   ⎥
⎢0  -cos(10)  -cos(10)  -cos(10)  0 ⎥
⎢                                   ⎥
⎣1     0         0         0      -1⎦

Rank of the Jacobian for angular velocity (Jw) at specific configuration:
2

Is the configuration singular? No
